In [1]:
import pandas as pd

# Function to downcast numeric columns to save memory
def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

# Load the CSV file using pandas
columns_to_use = [
    '70000', 'D', 'N', 'F', 'MK15 9HP', 'WILLEN', 'MILTON KEYNES', 
    'MILTON KEYNES.1', 'MILTON KEYNES.2', '1995-07-07 00:00', 'A'
]

df = pd.read_csv('C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/UK_property_price/UKarchive/202304.csv', usecols=columns_to_use)

# Rename the columns for ease of use
df = df.rename(columns={
    '70000': 'price', 
    'D': 'Property_Type', 
    'N': 'Old/New', 
    'F': 'Duration', 
    'MK15 9HP': 'Postcode', 
    'WILLEN': 'Locality', 
    'MILTON KEYNES': 'Town/City', 
    'MILTON KEYNES.1': 'District', 
    'MILTON KEYNES.2': 'County', 
    '1995-07-07 00:00': 'Date_of_Transfer', 
    'A': 'PPDCategory_Type'
})

# Apply downcasting to reduce memory usage immediately
df = downcast(df)

# Stratified sampling to reduce dataset size (75% sampling by 'Property_Type' and 'Town/City')
df_sampled = df.groupby(['Property_Type', 'Town/City'], group_keys=False).apply(lambda x: x.sample(frac=0.75, random_state=42)).reset_index(drop=True)

# Filter the dataset to include only standard residential sales (PPDCategory_Type 'A')
df_sampled = df_sampled[df_sampled['PPDCategory_Type'] == 'A']

df_sampled = downcast(df_sampled)


C:\Users\Guest01\AppData\Local\Temp\ipykernel_14660\3061195189.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby(['Property_Type', 'Town/City'], group_keys=False).apply(lambda x: x.sample(frac=0.75, random_state=42)).reset_index(drop=True)


In [2]:
# Handling missing values for categorical columns efficiently
categorical_columns = ['Property_Type', 'Old/New', 'Duration', 'Postcode', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']
mode_values = {col: df_sampled[col].mode()[0] for col in categorical_columns}
df_sampled = df_sampled.fillna(mode_values)

# Fill missing 'price' values (numeric) with the median
df_sampled['price'] = df_sampled['price'].fillna(df_sampled['price'].median())

# Extract date features
df_sampled['Date_of_Transfer'] = pd.to_datetime(df_sampled['Date_of_Transfer'])
df_sampled['Year'] = df_sampled['Date_of_Transfer'].dt.year
df_sampled['Month'] = df_sampled['Date_of_Transfer'].dt.month
df_sampled['Day'] = df_sampled['Date_of_Transfer'].dt.day
df_sampled['DayOfWeek'] = df_sampled['Date_of_Transfer'].dt.dayofweek


In [3]:
# Function to apply frequency encoding for categorical variables
def optimized_frequency_encoding(df, categorical_columns):
    for col in categorical_columns:
        if df[col].dtype == 'object':
            freq_encoding = df[col].value_counts().to_dict()
            df[col + '_freq'] = df[col].map(freq_encoding)
            df = df.drop(columns=[col])  # Drop the original object column
    return df

# List of categorical columns
categorical_columns = ['Postcode', 'Property_Type', 'Old/New', 'Duration', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']

# Apply frequency encoding to all categorical columns
df_sampled = optimized_frequency_encoding(df_sampled, categorical_columns)

# Unified downcast function for all numeric columns
def downcast_all_numeric(df):
    df[df.select_dtypes(include=['float64']).columns] = df.select_dtypes(include=['float64']).apply(pd.to_numeric, downcast='float')
    df[df.select_dtypes(include=['int64']).columns] = df.select_dtypes(include=['int64']).apply(pd.to_numeric, downcast='integer')
    return df

# Downcast all numeric columns (including frequency-encoded columns)
df_sampled = downcast_all_numeric(df_sampled)

In [4]:
# Define features (X) and target (y)
X = df_sampled.drop(columns=['price', 'Date_of_Transfer'])  # Drop the target 'price' and the raw date column
y = df_sampled['price']

# Check data types to ensure all columns are numeric
print(f"Data types in X:\n{X.dtypes}")
if X.select_dtypes(include=['object']).empty:
    print("All columns are numeric.")
else:
    print("Some columns are still non-numeric.")

# Print the shape of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Data types in X:
Year                     int32
Month                    int32
Day                      int32
DayOfWeek                int32
Postcode_freq            int16
Property_Type_freq       int32
Old/New_freq             int32
Duration_freq            int32
Locality_freq            int32
Town/City_freq           int32
District_freq            int32
County_freq              int32
PPDCategory_Type_freq    int32
dtype: object
All columns are numeric.
Shape of X: (20274118, 13)
Shape of y: (20274118,)


In [5]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")

In [11]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for XGBoost Regressor
def calculate_metrics(X_train, X_test, y_train, y_test):
    xgb_model = xgb.XGBRegressor(tree_method='hist', random_state=42)

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [30, 50],  # Number of boosting stages
        'learning_rate': [0.05, 0.1],  # Step size shrinkage
        'max_depth': [3, 5, 7],  # Maximum depth of individual trees
        'min_child_weight': [1, 2],  # Minimum sum of weights of all observations required in a child
        'subsample': [0.8, 0.9],  # Fraction of samples used for training
        'colsample_bytree': [0.8, 1.0],  # Fraction of features used per tree
        'tree_method': ['gpu_hist'],  # GPU accelerated method
        'max_bin': [256],  # Reduces memory usage by limiting bins
        'gpu_id': [0]  # Specifies the GPU ID (if you have multiple GPUs)
    }

    random_search = RandomizedSearchCV(xgb_model, param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, error_score='raise')

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }




In [12]:
import gc # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()    

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")

c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:41:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:41:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0.

Metrics for sample size 1.0:
RMSE: 176929.35361079557
MAPE: 0.4143910409510097
R2: 0.4972972869873047
nRMSE: 0.0019658817286297477
Execution Time (Raw): 1291.2802872657776
Normalized Time (s/MB): 1.152616949423308
Average CPU Usage: 56.349999999999994
Sample Size: 1.0
--------------------------------------------------


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:50:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:50:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-window

Metrics for sample size 0.5:
RMSE: 173934.66027837776
MAPE: 0.4620963521323687
R2: 0.5045651197433472
nRMSE: 0.0031059761318634623
Execution Time (Raw): 527.3418004512787
Normalized Time (s/MB): 0.9404855844045714
Average CPU Usage: 52.15
Sample Size: 0.5
--------------------------------------------------


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:53:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:53:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


...

c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:55:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:55:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Metrics for sample size 10000:
RMSE: 165360.38032155775
MAPE: 0.51166658861457
R2: 0.35683536529541016
nRMSE: 0.011858040897924542
Execution Time (Raw): 4.790900468826294
Normalized Time (s/MB): 10.826774245689656
Average CPU Usage: 57.650000000000006
Sample Size: 10000
--------------------------------------------------


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:55:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Metrics for sample size 100000:
RMSE: 152108.23648660487
MAPE: 0.4330607745794265
R2: 0.49570369720458984
nRMSE: 0.006914120746766647
Execution Time (Raw): 7.065438985824585
Normalized Time (s/MB): 1.5966917564655172
Average CPU Usage: 61.400000000000006
Sample Size: 100000
--------------------------------------------------


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:55:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [14]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df

Total Execution Time for Entire Process (Raw): 35 minutes and 2.18 seconds
Total Normalized Execution Time for Entire Process: 0.99796494 seconds per MB
Total Average CPU Usage for Entire Process: 57.13%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,176929.353611,0.414391,0.497297,0.001966,1291.280287,1.152617,56.35,1.000
1,173934.660278,0.462096,0.504565,0.003106,527.341800,0.940486,52.15,0.500
2,174174.634799,0.460195,0.503197,0.003264,208.580203,0.743983,55.25,0.250
3,174502.104401,0.464765,0.501327,0.003363,55.663649,0.397093,58.20,0.125
4,101459.824906,0.444166,0.380346,0.105522,3.481426,786.754256,58.15,100.000
5,114632.489781,0.444027,0.421793,0.054149,3.975189,89.833782,57.90,1000.000
6,165360.380322,0.511667,0.356835,0.011858,4.790900,10.826774,57.65,10000.000
7,152108.236487,0.433061,0.495704,0.006914,7.065439,1.596692,61.40,100000.000
